In [1]:
import json
import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4
import time
from time import mktime
import csv
import pdb
from datetime import timezone
from ifcb_helper import get_bins_in_range, get_datasets
from uptime import uptime
from dateutil.relativedelta import relativedelta
from scipy.io import loadmat

In [2]:
import platform
ptype=platform.system()

In [3]:
# path to data glider files
if ptype=='Windows':
    fpath='x:/glider_data/'
else:
    fpath='/cencoos_archive/glider_data/'
dir56='Line56'
dir67='Line67'
dirTr='Trinidad'

In [4]:
# need to have start times
thenow=dt.datetime.now(dt.UTC)

In [58]:
# get northernmost glider first
filetoload=fpath+dirTr+'/linux_trinidad_glider.txt'
fid=open(filetoload,'r')
lines=fid.readlines()
fid.close()
kl=len(lines)
# problem is that the most recent data isn't on the shared disk...
bigtime=[]
for theindex in np.arange(0,kl):
#for aline in lines:
    aline=lines[theindex]
    parts=aline.split('/')
    subpart=parts[-1].split(',')
    if theindex==kl-1:
        aline=lines[theindex-1]
        parts=aline.split('/')
    lp=len(parts)
    if ptype=='Windows':
    # we need to skip the first part of the output
        lp=len(parts)
        file_for_glider='x:'
        for kp in np.arange(2,lp-1):
            file_for_glider=file_for_glider+'/'+parts[kp]
        file_for_glider=file_for_glider+'/'+subpart[0]+'.mat'
        thedata=loadmat(file_for_glider)
    if ptype=='Linux':
        file_for_glider=''
        for kp in np.arange(0,lp-1):
            file_for_glider=file_for_glider+'/'+parts[kp]
        file_for_glider=file_for_glider+'/'+subpart[0]+'.mat'
        thedata=loadmat(file_for_glider)
    test=thedata['ttarr'][:,1] # this is the time array
# okay we have an issue in that how do we actually count days from profiles?
# keep track of start and stop of each deployment?
    deploystart=test[0]
    deploystop=test[-1]
    if theindex==0:
        bigtime=[[deploystart, deploystop]]
    else:
        bigtime=np.vstack((bigtime,[[deploystart, deploystop]]))
#    #bigtime=test
#else:

#    bigtime=np.append(bigtime,test)


In [59]:
bigtime

array([[735919.88769745, 736032.71432028],
       [736032.83696814,             nan],
       [736224.77306749, 736466.14049569],
       [736473.76974373, 736624.74304256],
       [736624.75543373, 736851.6901553 ],
       [736851.77409704, 737005.45664294],
       [737167.77288163, 737335.12789475],
       [737371.79175375, 737524.72296037],
       [737524.80692946, 737656.56881471],
       [737684.74975448, 737866.36485603],
       [737954.80160108, 737962.70549873],
       [738050.79117294, 738193.69046692],
       [738384.78551888, 738392.82330126],
       [738472.78422181, 738509.44371517],
       [738912.83801711, 739112.76183733],
       [739172.75696344, 739354.71038875],
       [739354.76824749, 739485.30161874]])

In [60]:
bigtime=pd.to_datetime(bigtime-719529,unit="D",utc=True) # trick to convert from stackoverflow
# 719529 is 1970-01-01 which is the origin date for to_datetime
#

In [61]:
# figure out start and stop dates for the quarter, 1/2 year, and ioos year
theyear=thenow.year
themonth=thenow.month
theday=thenow.day
if themonth < 4:
    quarter_start=1
elif themonth >=4 and themonth < 7:
    quarter_start=4
elif themonth >=7 and themonth < 10:
    quarter_start=7
elif themonth >=10:
    quarter_start=10
thequarter_start=dt.datetime(theyear,quarter_start,1,0,0,0,tzinfo=timezone.utc)
thequarter_end=dt.datetime(theyear,quarter_start+3,1,0,0,0,tzinfo=timezone.utc)
# we also want 6 months past
halfyear_start=dt.datetime(theyear,themonth,theday,0,0,0,tzinfo=timezone.utc)-relativedelta(months=6)
if themonth < 7:
    yeartouse=theyear-1
else:
    yeartouse=theyear
ioosyear_start=dt.datetime(yeartouse,7,1,0,0,0,tzinfo=timezone.utc)


In [73]:
ld=len(bigtime)
for ig in np.arange(0,ld):
    # this logic doesn't work
    # we have a case where start is 8/2 but glider start is 8/4 and glider end is 8/21 so last part returns False but that is not what we want
    #
    test1=thequarter_start >= bigtime[ig][0]
    test2=thequarter_start <= bigtime[ig][1]
    test3=thequarter_end >= bigtime[ig][0]
    test4=thequarter_end <= bigtime[ig][1]
    #
    # if test1==True and test2==False skip data set
    # if test1==True and test2==True use part of the data set
    # if test1==False and test2==True Need to check if end values are within the bounds?
    # we can not have the situation where test1==False and test2==False
    # if test3==False and test4==True skip data set
    # if test3==True and test4==False This can not happen?
    # if test3==True and test4==True use part of the data set
    # 




    # if value is between start and stop then both are test1=true,test2=true  visually this is [    +    ] 
    # if value is outside of range test1=true, test2=false  visually this is [     ] + 
    # if value is outside of start but with end then test1=false, test2=true  visually this is  + [      ]
    # for this case we would ideally have the first case happen also before this case.
    # I don't think we can have the situation where test1=false and test2=false
    # if we use + as start time and * as end times we could have the following possible scenarios
    # [  +    ][    ][    *  ] 
    # [    ] + [     *]  or [    ]+ [      ] *[     ] or [   ] + [    ][  *] or [  +   ][   ]*[   ]



    qtdays=(thequarter_start >= bigtime[ig][0])and(thequarter_start <= bigtime[ig][1])
    print(['quater days '+str(qtdays)])
    sixdays=(halfyear_start >= bigtime[ig][0])and(halfyear_start <= bigtime[ig][1])
    print(['sixdays '+str(sixdays)])
    yeardays=(ioosyear_start >= bigtime[ig][0])and(ioosyear_start <= bigtime[ig][1])
    print(['yeardays '+str(yeardays)])

['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays False']
['yeardays False']
['quater days False']
['sixdays True']
['yeardays False']
['quater days True']
['sixdays False']
['yeardays True']


In [63]:
halfyear_start

datetime.datetime(2024, 2, 21, 0, 0, tzinfo=datetime.timezone.utc)

In [64]:
bigtime[-1]

<DatetimeArray>
['2024-04-12 18:26:16.582999552+00:00', '2024-08-21 07:14:19.859002112+00:00']
Length: 2, dtype: datetime64[ns, UTC]

In [68]:
halfyear_start >= bigtime[-2][0]

True

In [69]:
halfyear_start <= bigtime[-2][1]

True

False

In [30]:
test=[[deploystart,deploystop]]
test=np.vstack((test,[[deploystart ,deploystop]]))
#test=np.append(test,[[deploystart] ,[deploystop]])

In [32]:
test

array([[739354.76824749, 739485.30161874],
       [739354.76824749, 739485.30161874]])

In [ ]:
len(test)

In [ ]:
test.shape